#### **모두를 위한 딥러닝**
slide
https://drive.google.com/drive/folders/1qVcF8-tx9LexdDT-IY6qOnHc8ekDoL03

git
https://github.com/deeplearningzerotoall/PyTorch?tab=readme-ov-file

youtube
https://www.youtube.com/watch?v=7eldOrjQVi0&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv


---

### Logistic Classification (Regression)




**Sigmoid**
- 단순 Linear Regression은 결과값이 무한대까지 커질 수 있다.
- 그러나 합/불 같은 여부를 판단하려면 0과 1사이의 확률이 필요하다.
    - 따라서, Sigmoid 함수를 사용한다.
$$ H(X) = \frac{1}{1+e^{-W^T X}} $$


**BCE (Binary Cross Entropy)**
- 정답이 0아니면 1이므로 MSE대신 `log`를 활용한 BCE 사용
    - 0~1사이 값이 나오므로 MSE는 오차제곱이므로 아무리커도 Loss가 1 이하이다
    - BCE는 log로 인해 값이 훨씬 크게나옴
    
    $$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 1. nn.Module로 모델 생성
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1) # input:8, output:1
        self.sigmoid = nn.Sigmoid()   # sigmoid: 0~1 사이 확률로 변환

    def forward(self, x):
        return self.sigmoid(self.linear(x))

# 2. 데이터 및 모델 초기화
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape, y_train.shape)

model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr=1)

torch.Size([759, 8]) torch.Size([759, 1])


In [2]:
# 3. 학습 루프
nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # fowardProp, BCE 계산
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train) # BCE 사용

    # backprop + step
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 로그 출력
    if epoch % 10 == 0:
        # 정확도 계산 과정
        prediction = hypothesis >= torch.tensor([0.5])     # 0.5 넘으면 True(1)
        correct_prediction = prediction.float() == y_train # 정답과 비교
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f} Accuracy {accuracy*100:2.2f}%')

Epoch    0/100 Cost: 0.724468 Accuracy 39.66%
Epoch   10/100 Cost: 0.566778 Accuracy 67.98%
Epoch   20/100 Cost: 0.534282 Accuracy 73.39%
Epoch   30/100 Cost: 0.515464 Accuracy 76.02%
Epoch   40/100 Cost: 0.503685 Accuracy 77.34%
Epoch   50/100 Cost: 0.495855 Accuracy 77.34%
Epoch   60/100 Cost: 0.490410 Accuracy 77.08%
Epoch   70/100 Cost: 0.486493 Accuracy 77.73%
Epoch   80/100 Cost: 0.483599 Accuracy 77.47%
Epoch   90/100 Cost: 0.481414 Accuracy 77.47%
Epoch  100/100 Cost: 0.479734 Accuracy 77.34%


---

### Softmax Classification

**Softmax**
- 어떤 숫자 벡터를 **확률 분포**로 변환
- 모든 출력값은 0~1 사이이며, 총합은 항상 **1**이 됨

$$P(class = i) = \frac{e^i}{\sum e^j}$$

### Cross Entropy
- 실제 정답 분포($P$)와 모델이 예측한 확률 분포($Q$) 사이의 거리를 측정하는 **손실 함수**.

$$H(P, Q) = -\sum_{x \in \mathcal{X}} P(x) \log Q(x)$$

$$L = \frac{1}{N} \sum -y \log(\hat{y})$$
- $N$: 데이터의 개수 (Batch size)
- $y$: 실제 정답 (0 또는 1)
- $\hat{y}$: 모델의 예측 확률
- 데이터 여러 개(N개)의 오차를 다 더해서 평균

In [3]:
# 모델생성
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

In [4]:
# dataloader
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

# y_data: 클래스 인덱스 (0, 1, 2 중 하나)
# F.cross_entropy를 쓸 때는 One-hot이 아닌 인덱스(LongTensor)를 넣어야 함
y_data = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_data)
# F.cross_entropy를 쓸때는 label값이 `정수`
y_train = torch.LongTensor(y_data)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [5]:

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} loss: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 loss: 1.882042
Epoch  100/1000 loss: 0.647751
Epoch  200/1000 loss: 0.566302
Epoch  300/1000 loss: 0.512771
Epoch  400/1000 loss: 0.468940
Epoch  500/1000 loss: 0.429913
Epoch  600/1000 loss: 0.393474
Epoch  700/1000 loss: 0.358183
Epoch  800/1000 loss: 0.322894
Epoch  900/1000 loss: 0.286837
Epoch 1000/1000 loss: 0.252270
